# Detecção automática de expressões multi-palavras

Este notebook mostra como detectar automaticamente frases comuns (aka multi-word expressions, word n-gram collocations) a partir de um stream de sentenças.

Paper interessante sobre o assunto: 
[Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)


Os dados utilizados são os atos normativos do TCU que foram pré-processados utilizando a seguinte análise:

lower case, remoção de acentos, stemming


## Inicialização

In [23]:
DOCS_FOLDER = 'Dados/atos-processados/'
BIGRAM_MODEL='bigram_model.bin'
TRIGRAM_MODEL='trigram_model.bin'

## Obtenção dos dados

In [24]:
import glob
import os

def find_files(folder, remove_empty = False):
    """
    Find all files in [folder]

    folder  :   string
                    folder to search (not recursive)
    """
    files =  glob.iglob(folder + '**/*.txt', recursive=True)
    return files

In [25]:
def get_sentences(input_file_pointer):
    sentences = []
    while True:
        line = input_file_pointer.readline()
        
        if not line:
            break
        
        processed_line = line.strip()
        
        if processed_line:
            sentences.append(processed_line)
        
    return sentences

In [26]:
# Diretório deste notebook
diretorio_notebook = os.getcwd()

# Diretório onde estão armazenados os dados
diretorio_dados = os.path.join(diretorio_notebook, os.path.pardir, DOCS_FOLDER)

files = find_files(diretorio_dados)

atos = []

for i, fpath in enumerate(files):
    with open(fpath, encoding="utf8") as f:
        sentences = get_sentences(f)
        atos.append(sentences)

In [27]:
from nltk.tokenize import word_tokenize

sentence_stream = []

for content in atos:
    sentenca_tokenizada = []
    for sentenca in content:
        sentenca_tokenizada = word_tokenize(sentenca)
        sentence_stream.append(sentenca_tokenizada)

# Treinamento do modelo

In [28]:
from gensim.models.phrases import Phrases, Phraser

biset = set()
triset = set()

bigram = Phraser(Phrases(sentence_stream, min_count=5))
trigram = Phraser(Phrases(bigram[sentence_stream], min_count=5))

for sent in sentence_stream:
    bigrams_ = [b for b in bigram[sent] if b.count('_') == 1]
    trigrams_ = [t for t in trigram[bigram[sent]] if t.count('_') == 2]

    biset.update(bigrams_)
    triset.update(trigrams_)

# Exemplos de bigramas e trigramas identificados

In [29]:
print(bigram['tribunal', 'cont', 'unia'])

['tribunal', 'cont_unia']


In [30]:
print(trigram[bigram['tribunal', 'cont', 'unia']])

['tribunal_cont_unia']


In [31]:
print(trigram[bigram['fund', 'particip', 'municipi']])

['fund_particip_municipi']


### Foram identificados 11719 bigramas: vejam exemplos

In [36]:
len(biset)

11719

In [38]:
cont = 0
for idx, x in enumerate(biset):
    print(x)
    if idx > 100:
        break

riach_santan
incidente_redutor
estad_acr
sergip_sescoops
atest_obit
consanguine_afins
fortal_taboca
estadual_distrital
terceiriz_estagiari
compativ_dem
carat_experimental
melhor_pratic
diretori_municip
remoca_ped
licencapremi_assidu
decisa_irrecorr
civil_fnac
uberland_ufu
regia_df
formaca_red
fiscal_comercial
imov_rur
totaliz_imov
gram_loureir
assent_funcional
relat_subitemns
lanc_corret
eduard_duailib
rodovi_federalm
siti_dabad
gratific_natalin
prevenca_detecca
portador_necess
citaca_revel
infraestrutur_seinfraoper
financi_nord
itaun_sul
conceit_desempenh
georeferenci_passiv
obtem_font
pedr_osori
mat_queim
doutor_severian
homolog_estagi
eventu_diferenc
beneficiariotitul_vbmt
licenc_conced
olh_dagu
carnaubeir_penh
obrig_comet
amaral_ferrador
comprovad_estudant
capa_lea
sebastia_parais
condutor_veicul
quinz_minut
lago_velh
independente_objetiv
cbex_mptcu
dissemin_incentiv
jos_safira
vincul_ministerial
uj_rfb
vint_cent
pitagor_fas
relaca_accountability
diurn_escal
cons_cct
formulari_gpd


### Foram identificados 8641 trigramas: vejam exemplos

In [33]:
len(triset)

8641

In [39]:
cont = 0
for idx, x in enumerate(triset):
    print(x)
    if idx > 100:
        break

sant_catarin_crasc
maior_absolut_membr
sucint_comissa_inquerit
nom_inscrica_oab
bem_quaisqu_outr
joa_rio_peix
licenc_mesm_espec
prev_pes_fator
procurador_region_unia
dad_text_imagens
quantidad_montant_repass
superintendente_regional_incr
ger_apoi_caix
incr_superintendente_regionalr
outr_norm_infraleg
ses_depart_regionalb
numer_professor_equivalent
certific_alt_gerenc
term_trat_constitut
grup_artilh_antiaere
nr_btcu_dou
leg_regiment_resolv
senac_administr_regionalm
estud_lingu_estrangeir
estudant_portador_necess
hospital_universitari_antoni
cas_afirmat_houv
ressalv_irregul_sintes
submet_deliber_colegi
fator_popul_colun
passagens_folh_pagament
encaminh_cop_pec
respect_are_atuaca
conhec_habil_atitud
estagi_folh_frequenc
secretariageral_presidente_segepr
nom_unidadesinstituico_papel
contab_legal_legitim
cifpmint_coeficient_individual
classific_licenc_sab
cont_sinal_negat
anuenc_dirigent_unidad
responsabil_usuari_tru
profer_primeir_lug
funca_poss_retard
lei_divers_tipolog
comit_olimp_brasil